In [1]:
from sklearn.model_selection import train_test_split
from clustering_alg import custom_clustering
# import gensim.models
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import neural_network
import pandas as pd
import numpy as np
import re
from sklearn import metrics
from sklearn.metrics import adjusted_rand_score
import gensim
from itertools import chain

X_train: (870, 34)
X_test: (373, 34)
Y_train: (870, 1)
Y_test: (373, 1)


In [2]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')
df_train.head()

,Id,Comment,Topic
0,0x840,A few things. You might have negative- frequen...,Biology
1,0xbf0,Is it so hard to believe that there exist part...,Physics
2,0x1dfc,There are bees,Biology
3,0xc7e,I'm a medication technician. And that's alot o...,Biology
4,0xbba,Cesium is such a pretty metal.,Chemistry


In [3]:
df_train.shape

(8695, 3)

In [4]:
# for comment in df_train.Comment:
#     df_train.loc[df_train['Comment']==comment,'Comment'] = comment.replace("\n"," ")
#     df_train.loc[df_train['Comment']==comment,'Comment'] = comment.replace("\\n"," ")
# df_train.iloc[0,1]

In [5]:
df_train

,Id,Comment,Topic
0,0x840,A few things. You might have negative- frequen...,Biology
1,0xbf0,Is it so hard to believe that there exist part...,Physics
2,0x1dfc,There are bees,Biology
3,0xc7e,I'm a medication technician. And that's alot o...,Biology
4,0xbba,Cesium is such a pretty metal.,Chemistry
...,...,...,...
8690,0x1e02,I make similar observations over the last week...,Biology
8691,0xc8d,You would know.,Biology
8692,0x723,Also use the correct number of sig figs,Chemistry
8693,0x667,"What about the ethical delimmas, groundbreaki...",Biology


In [6]:
df_train['Comment'].tolist()[0]

'A few things. You might have negative- frequency dependent selection going on where the least common phenotype, reflected by genotype, is going to have an advantage in the environment. For instance, if a prey animal such as a vole were to have a light and a dark phenotype, a predator might recognize the more common phenotype as food.  So if the light voles are more common, foxes may be keeping a closer eye out for light phenotypic voles, recognising them as good prey. This would reduce the light causing alleles due to increased predation and the dark genotypes would increase their proportion of the population until this scenario is reversed. This cycle continues perpetually. \\n\\nHowever, this is unlikely to be strictly yearly as it usually takes more time than a year for an entire populations allele frequencies to change enough to make a large enough difference to alter fitness. \\n\\nMore likely on a *year to year* basis, the population is experiencing fluctuating selection where a

In [7]:
comments_list = df_train['Comment'].tolist()
w2v_data = list()
for comment in comments_list:
    for sentence in sent_tokenize(comment):
        temp = []
        for word in word_tokenize(sentence):
            temp.append(word.lower())
        w2v_data.append(temp)    
w2v_data[0]

['a', 'few', 'things', '.']

In [8]:
df_train

,Id,Comment,Topic
0,0x840,A few things. You might have negative- frequen...,Biology
1,0xbf0,Is it so hard to believe that there exist part...,Physics
2,0x1dfc,There are bees,Biology
3,0xc7e,I'm a medication technician. And that's alot o...,Biology
4,0xbba,Cesium is such a pretty metal.,Chemistry
...,...,...,...
8690,0x1e02,I make similar observations over the last week...,Biology
8691,0xc8d,You would know.,Biology
8692,0x723,Also use the correct number of sig figs,Chemistry
8693,0x667,"What about the ethical delimmas, groundbreaki...",Biology


In [9]:
model = gensim.models.Word2Vec(w2v_data, min_count = 1, vector_size = 30, window = 5, hs=1)

In [20]:
import pickle

model = pickle.load(open("w2v.sav", 'rb'))
model.wv

In [21]:
my_dict = {}
for i in range(10):
    word = model.wv.index_to_key[i]
    my_dict[word] = model.wv[word]

my_dict

{'the': array([-0.8928743 , -1.4065392 ,  2.6949294 ,  0.6247602 , -0.47386536,
        -0.8281255 ,  1.7018446 ,  1.4255651 , -1.5988052 ,  0.03083327,
         0.74528885,  0.47732657, -1.7000575 , -0.78259325,  0.04934599,
        -0.7311268 ,  0.66943   ,  0.45484573,  0.15037026, -0.94900465,
         0.1786488 ,  1.8901759 ,  0.31204507,  1.7691251 ,  0.82538706,
         0.4895314 , -0.08870589,  0.8328206 ,  0.73379105, -1.2082686 ],
       dtype=float32),
 'to': array([-3.0631602 ,  0.5675518 ,  3.2337596 , -0.86805797, -0.93946326,
         0.21112137,  0.29869556,  1.4037484 , -0.29039633, -0.78236586,
         0.3477027 ,  0.53060234, -1.0156095 , -1.0663389 ,  0.919265  ,
        -2.9839041 ,  0.37433183,  1.4779383 , -0.5701126 ,  0.31992874,
        -1.3170632 , -1.3559316 , -1.8348321 ,  0.22616665,  0.5700524 ,
         1.2552073 ,  0.28103167, -0.45927277,  0.36451355, -1.738156  ],
       dtype=float32),
 'a': array([-1.1015084 , -1.12419   ,  2.5788941 ,  0.21873924

In [27]:
print(model.wv.similarity('cesium','observations'),model.wv.similarity('ride','car'))

0.17089069 0.5621512


In [15]:
words_vecs = pd.DataFrame(model.wv.vectors)
# clusters_count = 3

In [16]:
reduced_word_vecs = words_vecs.iloc[:200]
reduced_word_vecs.shape

(200, 100)

In [ ]:
words_labels = custom_clustering(words_vecs)

In [29]:
sample_labels_list = []
for i in range(6961):
    sample_labels_list.append(25)
for j in range(6961):
    sample_labels_list.append(400)
for k in range(6962):
    sample_labels_list.append(6432)
len(sample_labels_list)

20884

In [43]:
all_words_list = list(chain.from_iterable(w2v_data))
all_unique_words = list()
seen = set()
for word in all_words_list:
    if(word not in seen):
        all_unique_words.append(word)
    seen.add(word)
len(all_unique_words)

20884

In [57]:
normalized_df = pd.DataFrame(df_train)
for i in range(normalized_df.shape[0]):
    normalized_df.loc[i,'Comment'] = normalized_df.loc[i,'Comment'].lower()

In [60]:
for comment in normalized_df.Comment:
    for word in word_tokenize(comment):
        if(word in all_unique_words):
            normalized_df.loc[normalized_df['Comment']==comment,'Comment'] = comment.replace(word, all_unique_words[sample_labels_list[all_unique_words.index(word)]])
normalized_df

,Id,Comment,Topic
0,0x840,advantage advantage things. you might hadvanta...,Biology
1,0xbf0,advantage advantage so hard to believe that th...,Physics
2,0x1dfc,advantage advantage bees,Biology
3,0xc7e,advantageadvantage a medadvantagecatadvantageo...,Biology
4,0xbba,advantage advantage such a pretty metal.,Chemistry
...,...,...,...
8690,0x1e02,advantage advantage sadvantagemadvantagelar ob...,Biology
8691,0xc8d,advantage would know.,Biology
8692,0x723,advantage use the correct number of sig figs,Chemistry
8693,0x667,"advantage advantage the ethical delimmas, gro...",Biology


In [ ]:
#spliting data to train and test sets.
X_train, X_test, Y_train, Y_test = train_test_split(df['Comment'], df['ُTopic'], test_size=0.3, random_state=30)

X_train = pd.DataFrame(X_train, columns=['Comment'])
X_test = pd.DataFrame(X_test, columns=['Comment'])
Y_train = pd.DataFrame(Y_train, columns=['Topic'])
Y_test = pd.DataFrame(Y_test, columns=['Topic'])
print("X_train: " + str(X_train.shape))
print("X_test: " + str(X_test.shape))
print("Y_train: " + str(Y_train.shape))
print("Y_test: " + str(Y_test.shape))

In [271]:
train_docs = X_train['Comment'].tolist()
test_docs = X_test['Comment'].tolist()

for i in range(len(train_docs)):
    train_docs[i] = re.sub('[.,#$@!?"()]', '', str(train_docs[i]))

for i in range(len(test_docs)):
    test_docs[i] = re.sub('[.,#$@!?"()]', '', str(test_docs[i]))

-0.08860872 0.6494742


In [ ]:
vectorizer = TfidfVectorizer(max_df=0.99, min_df=0.01)
train_Tfidf = vectorizer.fit_transform(train_docs).toarray()

In [ ]:
feature_names = vectorizer.get_feature_names()

vocab = {}
for value, key in enumerate(feature_names):
    vocab[key] = value


In [ ]:
vectorizer = TfidfVectorizer(vocabulary=vocab)
test_Tfidf = vectorizer.fit_transform(test_docs).toarray()

In [ ]:
mlp = neural_network.MLPClassifier(hidden_layer_sizes=(128, 128, 32), learning_rate_init=0.001, max_iter=1000, tol=0.00001)
mlp.fit(train_Tfidf, np.array(Y_train['Topic'].tolist()))

In [ ]:
predicted_labels = mlp.predict(test_Tfidf)

In [ ]:
def purity_score(y_true, y_pred):
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

In [ ]:
randIndex = adjusted_rand_score(predicted_labels, np.array(Y_test['Topic'].tolist()))
print('rand index ', randIndex)
purity = purity_score(np.array(Y_test['Topic'].tolist()), predicted_labels)
print('purity ', purity)